# Interface creation

### Overview

This notebook helps create interface between two materials using Zur and McGill matching superlattices algorithm (according to https://doi.org/10.1063/1.3330840), where it searches for all possible superlattices for both materials and checks them for match within tolerances.
Pymatgen implementation of the algorithm is used here, where it is regarded as ZSL algorithm. (Documentation found in https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl)

### Steps to create an interface:

1.  Pass in materials from Materials Designer list by selecting input materials `materials_in` in the selector above:
    -   Material with index 0 regarded as a `Substrate`,
    -   Material with index 1 regarded as a `Layer`.
2.  Specify surface parameters as well as thickness in atomic layers for both materials (for 2D materials Miller indices (0, 0, 1) and thickness of 1 is assumed);

3.  Set the main parameter for the ZSL algorithm `MAX_AREA` -- maximum area of the interfaces search space in Angstroms^2;
4.  You can set fine-tuning parameters like `MAX_AREA_TOL`, `MAX_LENGTH_TOL`, etc if you want and know how to optimize search for your specific case;
5.  Add displacement of the layer relative to the substrate after the matching superlattice is found;
6.  Run cells below to:
    -   Load materials into JupyterLite environment,
    -   Execute the algorithm and find all matching lattices,
    -   Draw a scatter plot number of atoms vs strain for all found interfaces.
7.  Choose the index of the most optimal material for your task and run cell to `set_materials()`;
8.  Newly created interface should appear in the `materials_out` selector at the bottom of the dialog. Hit Submit to make it available it in the Materials Designer.


In [2]:
ZSL_GENERATOR_PARAMS = {
    # Main parameter:
    "MAX_AREA": 400,  # required
    # Fine-tuning parameters:
    "MAX_AREA_TOL": 0.09,  # optional
    "MAX_LENGTH_TOL": 0.03,  # optional
    "MAX_ANGLE_TOL": 0.01,  # optional
    "STRAIN_TOL": 10e-6,  # optional
}

INTERFACE_PARAMS = {
    "SUBSTRATE_INDEX": 0,
    "LAYER_INDEX": 1,
    "SUBSTRATE_MILLER": (1, 1, 1),
    "SUBSTRATE_THICKNESS": 1,
    "LAYER_MILLER": (0, 0, 1),
    "LAYER_THICKNESS": 1,
    "DISPLACEMENT_X": 0.0,
    "DISPLACEMENT_Y": 0.0,
    "DISPLACEMENT_Z": 3.0,
}

In [ ]:
get_materials()

In [ ]:
from development.pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from operator import itemgetter

pymatgen_materials = [to_pymatgen(item) for item in data]
for material in pymatgen_materials:
    print(material)


def create_interfaces(settings):
    # Create Interface Builder class
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["MAX_AREA_TOL"],
        max_area=settings["MAX_AREA"],
        max_length_tol=settings["MAX_LENGTH_TOL"],
        max_angle_tol=settings["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=pymatgen_materials[settings["SUBSTRATE_INDEX"]],
        film_structure=pymatgen_materials[settings["LAYER_INDEX"]],
        substrate_miller=settings["SUBSTRATE_MILLER"],
        film_miller=settings["LAYER_MILLER"],
        zslgen=zsl,
        strain_tol=settings["STRAIN_TOL"],
    )

    # Run the Interface Building process
    cib._find_terminations()
    matches = cib.zsl_matches
    terminations = cib.terminations

    # Create interfaces
    interfaces = []
    for termination in terminations:
        interfaces = list(
            cib.get_interfaces(
                termination,
                gap=settings["DISPLACEMENT_Z"],
                film_thickness=settings["LAYER_THICKNESS"],
                substrate_thickness=settings["SUBSTRATE_THICKNESS"],
                in_layers=True,
            )
        )

    print(f"Found {len(matches)} interfaces")
    print(f"Found {len(terminations)} terminations:", terminations)

    strain_modes = {
        "VON_MISES": "von_mises_strain",
        "STRAIN": "strain",
        "MEAN": "mean_abs_strain",
    }
    strain_mode = strain_modes["MEAN"]
    interfaces_list = list(interfaces)

    # Sort interfaces by ascending strain and then by ascending number of atoms
    sorted_interfaces = sorted(
        interfaces_list,
        key=lambda x: (itemgetter(strain_mode)(x), x["interface"].num_sites),
    )

    print("Interface with lowest strain (index 0):")
    print("    strain:", sorted_interfaces[0][strain_mode] * 100, "%")
    print("    number of atoms:", sorted_interfaces[0]["interface"].num_sites)
    PS = get_plot_settings()
    # plot stran vs number of atoms via matplotlib
    plot_strain_vs_atoms(strain_mode, sorted_interfaces, settings=PS)
    return sorted_interfaces


ZP = get_zsl_generator_params()
IP = get_interface_params()
interfaces = create_interfaces(settings={**ZP, **IP})

In [ ]:
m3_esse = from_pymatgen(interfaces[0]["interface"])
set_materials([m3_esse])